# Create Exercise Names Library

Let's create our library of exercise names so that users can log their exercises. We'll use Gemini to get the input exercise name from the JSON file, e.g. `air-squat`, and generate variations from that name.

In [23]:
from google import genai
from google.genai import types
import base64
import os

# Available Gemini models
gemini_2 = "gemini-2.0-flash-001"

# Low-bar safety filters
SAFETY_SETTINGS = [
    types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
    types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
    types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
    types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
]

## 1. Get the names from the JSON files

In [11]:
def get_json_filenames(folder_path):
    json_filenames = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            name = os.path.splitext(filename)[0]
            name = name.replace("'", "\'")
            json_filenames.append(name)
    json_filenames.sort()
    return json_filenames

# Example usage:
folder_path = 'data/jsons'
json_names = get_json_filenames(folder_path)
print(json_names)

['air-squat', 'assisted-chin-up', 'assisted-pull-up', 'back-extension', 'ball-slams', 'band-external-shoulder-rotation', 'band-internal-shoulder-rotation', 'band-pull-apart', 'banded-muscle-up', 'banded-side-kick', 'bar-dip', 'bar-hang', 'bar-muscle-up', 'barbell-curl', 'barbell-front-raise', 'barbell-hack-squat', 'barbell-lunge', 'barbell-lying-triceps-extension', 'barbell-preacher-curl', 'barbell-rear-delt-row', 'barbell-row', 'barbell-shrug', 'barbell-squat', 'barbell-standing-calf-raise', 'barbell-standing-triceps-extension', 'barbell-upright-row', 'barbell-walking-lunges', 'barbell-wrist-curl', 'barbell-wrist-curl-behind-the-back', 'barbell-wrist-extension', 'behind-the-neck-press', 'belt-squats', 'bench-dip', 'bench-press', 'bench-press-against-band', 'block-clean', 'block-snatch', 'board-press', 'body-weight-lunge', 'bodyweight-curl', 'bodyweight-leg-curl', 'box-squat', 'bulgarian-split-squat', 'cable-chest-press', 'cable-close-grip-seated-row', 'cable-crunch', 'cable-curl-with-

In [24]:
def generate(json_names):
  client = genai.Client(
      vertexai=True,
      project="gymally-test",
      location="us-central1",
  )

  text1 = types.Part.from_text(text=f"""
Given an input list of strength exercise names, you need to do this for each name:
1. Find out if the exercise name has popular variations (people calls it in different ways).
3. Generate another list that is a list of lists containing for each of the names of the input list:
    2.1. The input exercise name as-is
    2.2. That name properly (capitalize the first letter of each word, and separate words with a space).
    2.3. Potential aliases or variations of that name, properly written. Note that the name properly written (second item of each sublist) should always be also included as an alias. It is fine if an exercise name does not have variations or aliases (only the proper written name). Do not make up variations that are not popular.

Return only the list of lists. Do not return any code, only the list of lists.
                               
EXAMPLE:
Input List of names: [\"air-squat\", \"push-up\", \"deadlift\", \"mountain-climbers\"]
Output List of Lists: [[\"air-squat\", \"Air Squat\", [\"Air Squat\"]], [\"push-up\", \"Push-Up\", [\"Push-Up\", \"Push Up\", \"Pushup\", \"Press-Up\", \"Press Up\", \"Pressup\"]], [\"deadlift\", \"Deadlift\", [\"Deadlift\"]], [\"mountain-climbers\", \"Mountain Climbers\", [\"Mountain Climbers\", \"Running planks\", \"Spiders\"]]]

YOUR TURN:                             
Input List of names:
{json_names}
Output List of Lists:
""")

  model = gemini_2
  contents = [
    types.Content(
      role="user",
      parts=[
        text1
      ]
    )
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 0.0,
    top_p = 0.95,
    max_output_tokens = 8192,
    safety_settings = SAFETY_SETTINGS,
  )

  result = ""
  for chunk in client.models.generate_content_stream(
      model=model,
      contents=contents,
      config=generate_content_config,
  ):
      result += chunk.text

  return result

gen_exercise_list = generate(json_names)
print(gen_exercise_list)

```
[['air-squat', 'Air Squat', ['Air Squat']], ['assisted-chin-up', 'Assisted Chin-Up', ['Assisted Chin-Up']], ['assisted-pull-up', 'Assisted Pull-Up', ['Assisted Pull-Up']], ['back-extension', 'Back Extension', ['Back Extension', 'Hyperextension']], ['ball-slams', 'Ball Slams', ['Ball Slams']], ['band-external-shoulder-rotation', 'Band External Shoulder Rotation', ['Band External Shoulder Rotation']], ['band-internal-shoulder-rotation', 'Band Internal Shoulder Rotation', ['Band Internal Shoulder Rotation']], ['band-pull-apart', 'Band Pull-Apart', ['Band Pull-Apart']], ['banded-muscle-up', 'Banded Muscle-Up', ['Banded Muscle-Up']], ['banded-side-kick', 'Banded Side-Kick', ['Banded Side-Kick']], ['bar-dip', 'Bar Dip', ['Bar Dip']], ['bar-hang', 'Bar Hang', ['Bar Hang']], ['bar-muscle-up', 'Bar Muscle-Up', ['Bar Muscle-Up']], ['barbell-curl', 'Barbell Curl', ['Barbell Curl']], ['barbell-front-raise', 'Barbell Front-Raise', ['Barbell Front-Raise']], ['barbell-hack-squat', 'Barbell Hack-S

In [25]:
print(gen_exercise_list)

```
[['air-squat', 'Air Squat', ['Air Squat']], ['assisted-chin-up', 'Assisted Chin-Up', ['Assisted Chin-Up']], ['assisted-pull-up', 'Assisted Pull-Up', ['Assisted Pull-Up']], ['back-extension', 'Back Extension', ['Back Extension', 'Hyperextension']], ['ball-slams', 'Ball Slams', ['Ball Slams']], ['band-external-shoulder-rotation', 'Band External Shoulder Rotation', ['Band External Shoulder Rotation']], ['band-internal-shoulder-rotation', 'Band Internal Shoulder Rotation', ['Band Internal Shoulder Rotation']], ['band-pull-apart', 'Band Pull-Apart', ['Band Pull-Apart']], ['banded-muscle-up', 'Banded Muscle-Up', ['Banded Muscle-Up']], ['banded-side-kick', 'Banded Side-Kick', ['Banded Side-Kick']], ['bar-dip', 'Bar Dip', ['Bar Dip']], ['bar-hang', 'Bar Hang', ['Bar Hang']], ['bar-muscle-up', 'Bar Muscle-Up', ['Bar Muscle-Up']], ['barbell-curl', 'Barbell Curl', ['Barbell Curl']], ['barbell-front-raise', 'Barbell Front-Raise', ['Barbell Front-Raise']], ['barbell-hack-squat', 'Barbell Hack-S